###### Draw emotion flow

In [ ]:
colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]

names = ["anger", 	"anticipation", 	"disgust", 	"fear",  	"joy", 	"sadness", 	"surprise", 	"trust"]

In [ ]:
import plotly.graph_objects as go
from scipy.spatial import distance

# ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'trust']


def draw_ef(em):
  fig = go.Figure()
  count = 0




  x_axis = [str(i) for i in range(len(em.columns))]
  for i, row in em.iterrows():
    vector = list(row)
    # vector.pop(0)

    fig.add_trace(go.Scatter(x=x_axis, y=vector, name=names[count],
                          line=dict(color=colors[count], width=2, dash='dash')))
    
    # if count == 0:
    #   fig.add_trace(go.Scatter(x=x_axis, y=vector, name=names[count],
    #                           line=dict(color=colors[count], width=4)))
    # else:
    #   fig.add_trace(go.Scatter(x=x_axis, y=vector, name=names[count],
    #                           line=dict(color=colors[count], width=2, dash='dash')))
    count += 1
  fig.update_layout(title="'Masern: Erste Kindertagesstätten nehmen keine Kinder ohne Impfung auf'' / [Anger]",
                    xaxis_title='Chunks',
                    yaxis_title='Intensity',
                    showlegend=True,
                    #legend=dict(orientation="h")
                    )


  fig.show()


  # similarity_array = []
  # pred = em[str(0)].tolist() 
  # for i in range(len(em.columns)-1):
  #   curr = em[str(i+1)].tolist()
  #   dst = distance.euclidean(pred, curr)
  #   pred = curr 
  #   similarity_array.append(dst)
  # display("distances")  
  # display(similarity_array)

In [ ]:
import glob
import pandas as pd
import os

path = "/content/drive/MyDrive/emotion_flow_thesis/nrclexicon/data/EFs_lexicon_14000_CHeeSE_sentence_based/EFs/*.*"

highest_emotion_in_article_dict = {}

count = 0
for file in glob.glob(path):
    article_id = os.path.basename(os.path.normpath(file)) 
    count += 1
    print(count)
    
    
    article_ef_df = pd.read_csv(file, index_col=0)
    article_ef_df = article_ef_df.drop(['Positive', 'Negative'])
    draw_ef(article_ef_df)
    #display(article_ef_df)
    if count == 10:
      break

1


2


3


4


5


6


7


8


9


10


###### Expention / Reduction of emotion flows

In [ ]:
def get_shortest_distances(ef_df):
  similarity_array = []
  pred = ef_df[str(0)].tolist() 
  for i in range(len(ef_df.columns)-1):
    curr = ef_df[str(i+1)].tolist()
    dst = distance.euclidean(pred, curr)
    pred = curr 
    similarity_array.append(dst)
  
  min_el = min(similarity_array)
  min_elem_idx = [i for i, j in enumerate(similarity_array) if j == min_el]
  return min_elem_idx

In [ ]:
import statistics
def reduce_ef(reduce_from, reduce_to, ef_df):
  columns_to_reduce = reduce_from - reduce_to
  
  for i in range(columns_to_reduce):
    min_elem_idx = get_shortest_distances(ef_df)
    col_1 = str(min_elem_idx[0])
    col_2 = str(min_elem_idx[0] + 1)
    a = ef_df[col_1]
    a = list(a)
    b = ef_df[col_2]
    b = list(b)
    mean_vector = [statistics.mean([i, j]) for i, j in zip(a, b)]
    ef_df[col_1] = mean_vector
    ef_df.drop([col_2], axis='columns', inplace=True)
    ef_df.columns = [str(i) for i in range(len(ef_df.columns))]
  return ef_df

In [ ]:
import statistics
def expand_ef(expand_from, expand_to, ef_df):
  columns_to_reduce = expand_to - expand_from
  
  for i in range(columns_to_reduce):
    min_elem_idx = get_shortest_distances(ef_df)
    col_1 = str(min_elem_idx[0])
    col_2 = str(min_elem_idx[0] + 1)
    a = ef_df[col_1]
    a = list(a)
    b = ef_df[col_2]
    b = list(b)
    mean_vector = [statistics.mean([i, j]) for i, j in zip(a, b)]
    

    ef_df_1 = ef_df[[str(i) for i in range(int(col_1)+1)]] 
    ef_df_2 = pd.DataFrame(mean_vector, columns=['пох'])
    ef_df_2.index = ef_df_1.index
    ef_df_3 = ef_df[[str(i) for i in range(int(col_2), len(ef_df.columns))]]
    ef_df = pd.concat([ef_df_1, ef_df_2, ef_df_3], axis=1)
    ef_df.columns = [str(i) for i in range(len(ef_df.columns))]
  return ef_df
    
    


In [ ]:
import glob
import pandas as pd
import os


def normilize_emotion_flows(range_from, range_to, target_paragraphs_num):
  path = "/content/drive/MyDrive/emotion_flow_thesis/nrclexicon/data/EFs_lexicon_14000_CHeeSE_sentence_based/EFs/*.*"
  highest_emotion_in_article_dict = {}

  paragraph_range = range(range_from, range_to + 1)
  count = 0
  for file in glob.glob(path):
      article_id = os.path.basename(os.path.normpath(file)) 
      count += 1
      print(count)
     

      article_ef_df = pd.read_csv(file, index_col=0)
      num_pars = len(article_ef_df.columns)




      if num_pars in paragraph_range:
        article_ef_df = article_ef_df.drop(['Positive', 'Negative'])
        if num_pars > target_paragraphs_num: 
          article_ef_df = reduce_ef(num_pars, target_paragraphs_num, article_ef_df)
          print()
        elif num_pars < target_paragraphs_num:
          article_ef_df = expand_ef(num_pars, target_paragraphs_num, article_ef_df)
        else:
          # save ef like it is
          print()
        
        article_ef_df.to_csv("/content/drive/MyDrive/emotion_flow_thesis/nrclexicon/data/EFs_4_5_6_7_8_to_6/" + article_id, index=True)


      

In [ ]:
normilize_emotion_flows(4, 8, 6)

In [ ]:
path = "/content/drive/MyDrive/emotion_flow_thesis/nrclexicon/data/EFs_4_5_6_7_8_to_6/*.*"
  

count = 0
for file in glob.glob(path):
    article_id = os.path.basename(os.path.normpath(file)) 
    count += 1
    print(count)

In [ ]:
print(176+228+226+222+194)

1046


###### Create table with expended EFs

In [ ]:
import glob
import pandas as pd
import os
path = "/content/drive/MyDrive/emotion_flow_thesis/nrclexicon/data/EFs_4_5_6_7_8_to_6/*.*"


count = 0
df_expandend = pd.DataFrame()
for file in glob.glob(path):
    count += 1
    print(count)
    article_id = os.path.basename(os.path.normpath(file))
    article_ef_df = pd.read_csv(file, index_col=0)
    row = article_ef_df.values.flatten().tolist()
    
    entry_df = pd.DataFrame([[article_id] + row], columns=["article_id"] + list(range(len(row))))
    df_expandend = df_expandend.append(entry_df)
    
display(df_expandend)

In [ ]:
df_expandend.to_csv("/content/drive/MyDrive/emotion_flow_thesis/nrclexicon/data/EFs_4_5_6_7_8_to_6/one_vector_ef.csv", index=True)

###### Prepare database for vectors by topic

In [ ]:
import pandas as pd

topic_template_df = pd.DataFrame(columns=['article_id', "0", "1", "2", "3", "4", "5", "6", "7",])

topic_list = {'Arbeit/Soziales': topic_template_df.copy(),
 'Außenpolitik': topic_template_df.copy()}

topic_list['Arbeit/Soziales'].loc[len(topic_list['Arbeit/Soziales'].index)] = ["id.123", 0, 1, 2, 3, 4, 5, 6, 7]
topic_list['Außenpolitik'].loc[len(topic_list['Außenpolitik'].index)] = ["id.123", 7, 6, 5, 4, 3, 2, 1, 0]

display(topic_list['Arbeit/Soziales'])
display(topic_list['Außenpolitik'])


,article_id,0,1,2,3,4,5,6,7
0,id.123,0,1,2,3,4,5,6,7


,article_id,0,1,2,3,4,5,6,7
0,id.123,7,6,5,4,3,2,1,0


In [ ]:

def get_topic_dict():
  topic_list = {'Arbeit/Soziales': topic_template_df.copy(),
   'Außenpolitik': topic_template_df.copy(),
   'Digitalisierung': topic_template_df.copy(),
   'Einwanderung': topic_template_df.copy(),
   'Erziehung/Ausbildung': topic_template_df.copy(),
   'Finanzen': topic_template_df.copy(),
   'Freizeit/Modernes Leben': topic_template_df.copy(),
   'Gesellschaft': topic_template_df.copy(),
   'Gesundheitspflege': topic_template_df.copy(),
   'Internationale Organisation': topic_template_df.copy(),
   'Justiz': topic_template_df.copy(),
   'Justiz/Kriminalität': topic_template_df.copy(),
   'Katastrophe/Unglück': topic_template_df.copy(),
   'Krise/Krieg/Konflikt': topic_template_df.copy(),
   'Kultur/Kunst/Unterhaltung': topic_template_df.copy(),
   'Medizin/Gesundheit': topic_template_df.copy(),
   'Politik': topic_template_df.copy(),
   'Religion/Weltanschauung': topic_template_df.copy(),
   'Sicherheit': topic_template_df.copy(),
   'Sport': topic_template_df.copy(),
   'Umwelt': topic_template_df.copy(),
   'Wirtschaft': topic_template_df.copy(),
   'Wirtschaft/Finanzen': topic_template_df.copy(),
   'Wissenschaft, Technik': topic_template_df.copy(),
   'Wissenschaft/Technik': topic_template_df.copy()}
  return topic_list
 

In [ ]:

def get_topic_dict():
  topic_list = {'Arbeit/Soziales': [],
   'Außenpolitik': [],
   'Digitalisierung': [],
   'Einwanderung': [],
   'Erziehung/Ausbildung': [],
   'Finanzen': [],
   'Freizeit/Modernes Leben': [],
   'Gesellschaft': [],
   'Gesundheitspflege': [],
   'Internationale Organisation': [],
   'Justiz': [],
   'Justiz/Kriminalität': [],
   'Katastrophe/Unglück': [],
   'Krise/Krieg/Konflikt': [],
   'Kultur/Kunst/Unterhaltung': [],
   'Medizin/Gesundheit': [],
   'Politik': [],
   'Religion/Weltanschauung': [],
   'Sicherheit': [],
   'Sport': [],
   'Umwelt': [],
   'Wirtschaft': [],
   'Wirtschaft/Finanzen': [],
   'Wissenschaft, Technik': [],
   'Wissenschaft/Technik': []}
  return topic_list
 

In [ ]:
topic_list = get_topic_dict()
topic_list['Außenpolitik']



[]

In [ ]:
import json
import numpy as np
%cd 
path = '/content/drive/MyDrive/emotion_flow_thesis/nrclexicon/data/CHeeSE.json'
f = open(path,)

/root


In [ ]:
def add_to_dataframe_of_vectors_pertopic(article_jsn):
  list_general_area_of_interest = []
  article_id = article_jsn['article_id']
  for article_stance in article_jsn["article_stance"]:
    area_of_intereset = article_stance["general_area_of_interest"]
    #display(area_of_intereset)
    
    
    # if area_of_interest is already encountered --> skip the adding of vector to a corresponding database (it is already there) 
    if area_of_intereset in list_general_area_of_interest:  
      continue

    #corresponding_row = cheese_article_emotion_vectors.loc[cheese_article_emotion_vectors['article_id'] == article_id]
    topic_list[area_of_intereset].append(article_id)
    list_general_area_of_interest.append(area_of_intereset)
    
  # display(topic_list['Gesellschaft'])
    
    

In [ ]:
def inerate_via_articles(f):
  count = 0
  for line in f:
    jsn = json.loads(line)
    #display(jsn)
    
    
    add_to_dataframe_of_vectors_pertopic(jsn)
    

    count += 1
    # if count == 10:
      # break

    

In [ ]:
topic_list = get_topic_dict()
inerate_via_articles(f)

###### Distribution of topics for articles (by areas of interest)

In [ ]:
list(topic_list.keys())

In [ ]:
display(topic_list)

In [ ]:
## general distribution
import plotly.express as px
## проверяем какие топики в кластаре 1

#topics = cheese_ids_department['department'].unique()
topics = list(topic_list.keys())
print(topics)
# initialize bins ##
def init_bins_topics(): 
  bins_topics = {}
  for i in range(len(topics)):
    key = topics[i]
    bins_topics[key] = len(topic_list[key])
  return bins_topics

# claster level

bins_topics = init_bins_topics()


display(bins_topics)



# for i in range(len(identified_clusters)):
#   if "escenic" in data.index[i]:
#     continue
  
#   article_id = data.index[i].replace(".csv", "")
#   #print(article_id)
#   topic = cheese_ids_department.loc[article_id][0]
#   bins_topics[topic] += 1


print("--------")
print("--------")
bins_topics = dict(sorted(bins_topics.items(), key=lambda item: item[1]))
keys = pd.DataFrame(bins_topics.keys())
values = pd.DataFrame(bins_topics.values())
data_to_plot = pd.concat([keys, values], axis=1)
data_to_plot.columns = ['paragraphs_per_article', 'number']
fig = px.bar(data_to_plot, x='paragraphs_per_article', y='number')
fig.show()

['Arbeit/Soziales', 'Außenpolitik', 'Digitalisierung', 'Einwanderung', 'Erziehung/Ausbildung', 'Finanzen', 'Freizeit/Modernes Leben', 'Gesellschaft', 'Gesundheitspflege', 'Internationale Organisation', 'Justiz', 'Justiz/Kriminalität', 'Katastrophe/Unglück', 'Krise/Krieg/Konflikt', 'Kultur/Kunst/Unterhaltung', 'Medizin/Gesundheit', 'Politik', 'Religion/Weltanschauung', 'Sicherheit', 'Sport', 'Umwelt', 'Wirtschaft', 'Wirtschaft/Finanzen', 'Wissenschaft, Technik', 'Wissenschaft/Technik']


{'Arbeit/Soziales': 190,
 'Außenpolitik': 55,
 'Digitalisierung': 45,
 'Einwanderung': 56,
 'Erziehung/Ausbildung': 82,
 'Finanzen': 63,
 'Freizeit/Modernes Leben': 89,
 'Gesellschaft': 495,
 'Gesundheitspflege': 21,
 'Internationale Organisation': 21,
 'Justiz': 32,
 'Justiz/Kriminalität': 97,
 'Katastrophe/Unglück': 43,
 'Krise/Krieg/Konflikt': 19,
 'Kultur/Kunst/Unterhaltung': 44,
 'Medizin/Gesundheit': 139,
 'Politik': 212,
 'Religion/Weltanschauung': 143,
 'Sicherheit': 36,
 'Sport': 79,
 'Umwelt': 313,
 'Wirtschaft': 96,
 'Wirtschaft/Finanzen': 132,
 'Wissenschaft, Technik': 218,
 'Wissenschaft/Technik': 144}

--------
--------


###### K-Means Clustering

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.cluster import KMeans

data = pd.read_csv('/content/drive/MyDrive/emotion_flow_thesis/nrclexicon/data/EFs_4_5_6_7_8_to_6/one_vector_ef.csv', index_col=1)
data.drop(['Unnamed: 0'], axis=1, inplace=True)
display(data)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
article_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
escenic~blick~00007380219.csv,0.250000,0.00,0.500,0.000000,0.000000,0.000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.250000,0.0,0.500000,0.000000,0.000000,0.00,0.250000,0.00,0.500,0.000000,0.000000,1.000000,0.000000,0.000000,0.00,0.0,0.333333,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.666667,1.000000
escenic~blick~00005807440.csv,0.500000,0.00,0.500,0.375000,0.250000,0.000,0.0,1.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.00,1.500000,0.00,1.500,1.125000,0.750000,0.000000,0.000000,1.000000,0.00,0.0,0.000000,0.000000,1.000000,0.500000,0.500000,0.375000,0.250000,0.00,0.000000,0.000,0.50,0.375,0.250000,0.000000,0.500000,0.500000,0.500000,0.375000,0.250000,0.000000
escenic~blick~00006601650.csv,0.000000,0.00,0.250,0.000000,0.000000,0.000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.333333,0.000000,0.25,0.0,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.333333,0.000000,0.250000,0.000000,1.000000,0.250000
ld.1366933.csv,0.250000,0.00,0.000,0.166667,0.166667,0.125,0.5,0.00,0.000000,0.000000,0.500000,0.250000,0.250000,0.0,0.000000,0.166667,0.166667,0.25,0.250000,0.00,0.000,0.166667,0.333333,0.250000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.250000,0.000000,0.000000,0.333333,0.500000,0.25,0.000000,0.000,0.00,0.000,0.166667,0.000000,0.250000,0.500000,0.000000,0.000000,0.500000,0.250000
escenic~blick~00003404254.csv,0.500000,0.25,0.000,0.000000,0.000000,0.000,0.0,0.25,0.000000,0.500000,0.000000,0.142857,1.000000,0.0,0.000000,0.000000,0.000000,0.00,0.500000,0.25,0.000,0.000000,0.000000,0.142857,0.000000,0.250000,0.00,0.0,0.000000,0.142857,0.000000,0.250000,0.000000,0.500000,0.000000,0.00,0.000000,0.250,0.00,0.000,0.000000,0.142857,1.500000,0.500000,0.000000,0.500000,0.000000,0.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
escenic~blick~00008159488.csv,0.333333,0.00,0.000,0.000000,0.000000,0.000,0.0,1.50,0.000000,0.000000,0.000000,0.500000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,1.00,0.000,0.000000,0.000000,0.000000,0.000000,1.000000,0.00,0.0,0.000000,1.000000,0.333333,1.500000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.333333,0.500000,0.500000,0.250000,0.000000,2.000000
ld.1317620.csv,0.857143,0.00,0.000,0.000000,0.428571,0.000,0.0,0.00,0.000000,0.000000,0.428571,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.571429,0.20,0.100,0.000000,0.000000,0.250000,0.142857,0.000000,0.00,0.0,0.142857,0.000000,0.428571,0.200000,0.100000,0.000000,0.142857,0.25,0.285714,0.200,0.10,0.000,0.142857,0.250000,0.285714,0.000000,0.125000,0.250000,0.857143,0.250000
escenic~blick~00015018562.csv,0.571429,0.00,0.125,0.250000,0.500000,0.000,0.0,0.00,0.083333,0.166667,0.333333,0.000000,0.428571,0.0,0.083333,0.166667,0.333333,0.00,0.571429,0.00,0.125,0.250000,0.500000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.666667,0.285714,0.166667,0.166667,0.166667,0.166667,0.00,0.000000,0.000,0.00,0.000,0.000000,0.000000,0.142857,0.166667,0.125000,0.083333,0.000000,0.000000


In [ ]:
kmeans = KMeans(25)
kmeans.fit(data)

KMeans(n_clusters=25)

In [ ]:
identified_clusters = kmeans.fit_predict(data)
identified_clusters

array([19, 18, 17, ..., 19, 23, 19], dtype=int32)

In [ ]:
data.index[1]

'escenic~blick~00005807440.csv'

In [ ]:
for i in range(len(identified_clusters)):
  print(identified_clusters[i])
  if identified_clusters[i] == 2:
    path = "/content/drive/MyDrive/emotion_flow_thesis/nrclexicon/data/EFs_4_5_6_7_8_to_6/EFs/" + data.index[i]
    target_ef = pd.read_csv(path, index_col=0)
    draw_ef(target_ef)
  if i == 20:
    break
  


In [ ]:
from matplotlib import pyplot as plt

wcss=[]
for i in range(1,200):
  kmeans = KMeans(i)
  kmeans.fit(data)
  wcss_iter = kmeans.inertia_
  wcss.append(wcss_iter)

number_clusters = range(1,200)
plt.plot(number_clusters,wcss, size=20)
plt.title('The Elbow title')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')

NameError: ignored

In [ ]:
## other clustering approach
!pip install tslearn
from tslearn.clustering import TimeSeriesKMeans

km = TimeSeriesKMeans(n_clusters=5, metric="dtw")
labels = km.fit_predict(data)
display(labels)

km_bis = TimeSeriesKMeans(n_clusters=5, metric="softdtw")
labels_bis = km_bis.fit_predict(data)
display(labels_bis)

array([0, 3, 1, ..., 1, 2, 1])

array([4, 1, 4, ..., 4, 4, 4])

In [ ]:
count = 0
for index, row in data.iterrows():
  #escenic
  if "escenic" in index:
    continue
  count += 1
  print(count)


In [ ]:
path = "/content/drive/MyDrive/emotion_flow_thesis/nrclexicon/data/EFs_4_5_6_7_8_to_6/cheese_ids_department.csv"
cheese_ids_department = pd.read_csv(path, index_col=0)
cheese_ids_department

,department
article_id,
ld.1335892,International
ld.1423922,Meinung
ld.1488102,Schweiz
ld.1418266,Meinung
ld.1464431,International
...,...
ld.1401319,Wissen
ld.1396440,Wissen
ld.1433666,Hintergrund


In [ ]:
count = 0
for article_id, department in cheese_ids_department.iterrows():
  if "escenic" in article_id:
    continue
  count += 1
  print(count)

In [ ]:
len(identified_clusters)

In [ ]:
import plotly.express as px
## проверяем какие топики в кластаре 1

topics = list(topic_list.keys())

# initialize bins ##
def init_bins_topics(): 
  bins_topics = {}
  for i in range(len(topics)):
    key = topics[i]
    bins_topics[key] = 0
  return bins_topics

# claster level
for j in range(25):
  bins_topics = init_bins_topics()

  for i in range(len(identified_clusters)):

    # if "escenic" in data.index[i]:
    #   continue

    if identified_clusters[i] == j:
      article_id = data.index[i].replace(".csv", "")
  
      topics_in_article = [topic for topic in list(topic_list.keys()) if article_id in topic_list[topic]]
  

      for topic in topics_in_article:
        bins_topics[topic] += 1

  print("--------")
  print("--------")
  print("number of entries in cluster")
  print(sum(list(bins_topics.values())))
  keys = pd.DataFrame(bins_topics.keys())
  values = pd.DataFrame(bins_topics.values())
  data_to_plot = pd.concat([keys, values], axis=1)
  data_to_plot.columns = ['paragraphs_per_article', 'number']
  fig = px.bar(data_to_plot, x='paragraphs_per_article', y='number')
  fig.show()

--------
--------
number of entries in cluster
107


--------
--------
number of entries in cluster
88


--------
--------
number of entries in cluster
7


--------
--------
number of entries in cluster
378


--------
--------
number of entries in cluster
47


...

--------
--------
number of entries in cluster
21


--------
--------
number of entries in cluster
113


--------
--------
number of entries in cluster
1


--------
--------
number of entries in cluster
88


--------
--------
number of entries in cluster
85


In [ ]:
## general distribution
import plotly.express as px
## проверяем какие топики в кластаре 1

topics = cheese_ids_department['department'].unique()

# initialize bins ##
def init_bins_topics(): 
  bins_topics = {}
  for i in range(len(topics)):
    key = topics[i]
    bins_topics[key] = 0
  return bins_topics

# claster level

bins_topics = init_bins_topics()
for i in range(len(identified_clusters)):
  if "escenic" in data.index[i]:
    continue
  
  article_id = data.index[i].replace(".csv", "")
  #print(article_id)
  topic = cheese_ids_department.loc[article_id][0]
  bins_topics[topic] += 1
print("--------")
print("--------")
keys = pd.DataFrame(bins_topics.keys())
values = pd.DataFrame(bins_topics.values())
data_to_plot = pd.concat([keys, values], axis=1)
data_to_plot.columns = ['paragraphs_per_article', 'number']
fig = px.bar(data_to_plot, x='paragraphs_per_article', y='number')
fig.show()

--------
--------


###### Dimentionality reduction

In [ ]:
###### Data loader
def get_data():
  import glob
  import pandas as pd
  import os
  import numpy as np
  %cd
  path = "/content/drive/MyDrive/emotion_flow_thesis/nrclexicon/data/EFs_4_5_6_7_8_to_6/EFs/*.*"


  count = 0
  df_expandend = pd.DataFrame()
  data = np.empty((1046, 8, 6))
  display(data)
  for file in glob.glob(path):
    print(count)
    article_id = os.path.basename(os.path.normpath(file))
    article_ef_df = pd.read_csv(file, index_col=0)
    ef_values = article_ef_df.values    
    data[count] = ef_values
    count += 1
  return data
data = get_data()

In [ ]:
data.shape

(1046, 8, 6)

In [ ]:
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras import Input
from keras import Model


inp = Input(shape=(data.shape[1], data.shape[2]))
    
encoder = TimeDistributed(Dense(200, activation='tanh'))(inp)
encoder = TimeDistributed(Dense(50, activation='tanh'))(encoder)
latent = TimeDistributed(Dense(10, activation='tanh'))(encoder)
decoder = TimeDistributed(Dense(50, activation='tanh'))(latent)
decoder = TimeDistributed(Dense(200, activation='tanh'))(decoder)
out = TimeDistributed(Dense(3))(decoder)
autoencoder = Model(inputs=inp, outputs=out)
autoencoder.compile(optimizer='adam', loss='mse')

In [ ]:
data[0].shape

(8, 6)

In [ ]:
pred = autoencoder.predict(data)

In [ ]:
pred.shape

(1046, 8, 3)

###### Draw emotion flows by "area of interest"

In [ ]:
# for i in range(len(identified_clusters)):
#   print(identified_clusters[i])
#   if identified_clusters[i] == 1:
#     path = "/content/drive/MyDrive/emotion_flow_thesis/nrclexicon/data/EFs_4_5_6_7_8_to_6/EFs/" + data.index[i]
#     target_ef = pd.read_csv(path, index_col=0)
#     draw_ef(target_ef)
#   if i == 20:
#     break

import glob
import pandas as pd
import os

path = "/content/drive/MyDrive/emotion_flow_thesis/nrclexicon/data/EFs_4_5_6_7_8_to_6/EFs/*.*"
paths = glob.glob(path)

%cd
count = 0
for article_id in topic_list['Krise/Krieg/Konflikt']:
  path = "/content/drive/MyDrive/emotion_flow_thesis/nrclexicon/data/EFs_4_5_6_7_8_to_6/EFs/" + article_id + ".csv"
  if not (path in paths):
    continue  
  target_ef = pd.read_csv(path, index_col=0)
  draw_ef(target_ef)
  
  count += 1
  if count == 10:
    break

/root
